In [1]:
# Import libraries
import pandas as pd
import numpy as np
import cpi
import datetime as dt
import matplotlib.pyplot as plt

# Libraries for regression
import statsmodels.formula.api as sm
import statsmodels.api as sm1
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.api import add_constant
from linearmodels.iv import IV2SLS
from linearmodels.iv import compare

In [2]:
# Read files
execu500 = pd.read_csv('/Users/ASUS/Downloads/exesp500fullvariables.csv')
execu400 = pd.read_csv('/Users/ASUS/Downloads/exesp400fullvariables.csv')
execu600 = pd.read_csv('/Users/ASUS/Downloads/exesp600fullvariables.csv')
sp500 = pd.read_csv('/Users/ASUS/Downloads/sp500afull.csv')
sp400 = pd.read_csv('/Users/ASUS/Downloads/sp400afull.csv')
sp600 = pd.read_csv('/Users/ASUS/Downloads/sp600afull.csv')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\3539302744.py:2: DtypeWarning: Columns (77,80,96) have mixed types. Specify dtype option on import or set low_memory=False.
  execu500 = pd.read_csv('/Users/ASUS/Downloads/exesp500fullvariables.csv')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\3539302744.py:3: DtypeWarning: Columns (77,80,96) have mixed types. Specify dtype option on import or set low_memory=False.
  execu400 = pd.read_csv('/Users/ASUS/Downloads/exesp400fullvariables.csv')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\3539302744.py:4: DtypeWarning: Columns (77,80,96) have mixed types. Specify dtype option on import or set low_memory=False.
  execu600 = pd.read_csv('/Users/ASUS/Downloads/exesp600fullvariables.csv')
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\3539302744.py:5: DtypeWarning: Columns (26,30,969,970,980) have mixed types. Specify dtype option on import or set low_memory=False.
  sp500 = pd.read_csv('/Users/ASUS/Downloads/sp500afull.csv')
C:\Users\

In [3]:
# Rename key columns
sp500 = sp500.rename(columns={"fyear": "YEAR", "tic": "TICKER"})
sp400 = sp500.rename(columns={"fyear": "YEAR", "tic": "TICKER"})
sp600 = sp500.rename(columns={"fyear": "YEAR", "tic": "TICKER"})

In [4]:
# Combine executive data
df = pd.concat([execu500, execu400, execu600])
df['stockoptions'] = df['STOCK_AWARDS'] + df['OPTION_AWARDS']

# Adjust Stock Options based on CPI, base year is 2022
df["adjusted_stockoptions"] = df.apply(lambda x: cpi.inflate(x["stockoptions"], x["YEAR"]), axis=1)

# Adjust total compensation based on CPI, base year is 2022
df["adjusted_TC"] = df.apply(lambda x: cpi.inflate(x["TOTAL_SEC"], x["YEAR"]), axis=1)

# Change to million USD
df['adjusted_stockoptions'] = df['adjusted_stockoptions'] / 1000
df['adjusted_TC'] = df['adjusted_TC'] / 1000

In [5]:
# Drop rows with negative compensation
df = df[(df['adjusted_stockoptions']>0) & (df['adjusted_TC']>0)]

In [6]:
# Keep only CEOs data
ceo = df[(df['CEOANN'] == 'CEO')]

In [7]:
# Get the year that executive became CEO
ceo['dateceo'] = pd.to_datetime(ceo['BECAMECEO'])
ceo['yearceo'] = ceo['dateceo'].dt.year

# Get the tenure of CEOs - defined as : year of data observation - yearbecame ceo
ceo['tenure'] = (ceo['YEAR'].astype(float) - ceo['yearceo'])

# Drop negative tenures
ceo_tenure = ceo[(ceo['tenure'] > 0)]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\2882941134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceo['dateceo'] = pd.to_datetime(ceo['BECAMECEO'])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\2882941134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceo['yearceo'] = ceo['dateceo'].dt.year
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\2882941134.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [8]:
''' Detect outliers in compensation '''

''' Detection '''
# IQR
# Calculate the upper and lower limits
Q1a = ceo_tenure['adjusted_TC'].quantile(0.25)
Q3a = ceo_tenure['adjusted_TC'].quantile(0.75)
IQRa = Q3a - Q1a
lowera = Q1a - 1.5*IQRa
uppera = Q3a + 1.5*IQRa

# IQR
# Calculate the upper and lower limits
Q1b = ceo_tenure['adjusted_stockoptions'].quantile(0.25)
Q3b = ceo_tenure['adjusted_stockoptions'].quantile(0.75)
IQRb = Q3b - Q1b
lowerb = Q1b - 1.5*IQRb
upperb = Q3b + 1.5*IQRb
 
# Removing the outliers
ceo_tenure = ceo_tenure.loc[(ceo_tenure['adjusted_stockoptions']< upperb) & (ceo_tenure['adjusted_stockoptions'] > lowerb) 
                            & (ceo_tenure['adjusted_TC'] < uppera) & (ceo_tenure['adjusted_TC'] > lowera) ]

In [9]:
''' Detect outliers in tenure '''
# IQR
# Calculate the upper and lower limits
Q1_t = ceo_tenure['tenure'].quantile(0.25)
Q3_t = ceo_tenure['tenure'].quantile(0.75)
IQR_t = Q3_t - Q1_t
lower_t = Q1_t - 1.5*IQR_t
upper_t = Q3_t + 1.5*IQR_t
 
# Removing the outliers
ceo_tenure = ceo_tenure.loc[(ceo_tenure['tenure']< upper_t) & (ceo_tenure['tenure'] > lower_t) ]

In [13]:
# Combine financial data
df2 = pd.concat([sp500, sp400, sp600])

In [14]:
# Component of Altman's Z-score for public companies
df2['X1'] = (df2['act'] - df2['lct']) / df2['at']
df2['X2'] = df2['re'] / df2['at']
df2['X3'] = df2['ebit'] / df2['at']
df2['X4'] = df2['prcc_f'] * df2['csho'] / df2['lt']
df2['X5'] = df2['sale'] / df2['at']

# Calculate Z-score
df2['zscore'] = 1.2*df2['X1'] + 1.4*df2['X2'] + 3.3*df2['X3'] + 0.6*df2['X4'] + 1.0*df2['X5']

In [31]:
df2['emp'].describe()

count    46404.000000
mean        43.040830
std        104.980433
min          0.000000
25%          5.000000
50%         14.577000
75%         43.000000
max       2300.000000
Name: emp, dtype: float64

In [15]:
# Merge executive data with firm's financial data
secondstage = ceo_tenure.merge(df2, on = ['YEAR', 'TICKER'], how = 'left')
secondstage.head()

,EXEC_FULLNAME,CFOANN,EXECDIR,OLD_DATAFMT_FLAG,INTERLOCK,REPRICE,EXECRANK,CO_PER_ROL,CONAME,TITLEANN,...,stko,weburl,dldte,ipodate,X1,X2,X3,X4,X5,zscore
0,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,0.0,www.aa.com,NaN,NaN,-0.055001,-0.109281,0.036370,0.225802,0.774164,0.810671
1,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,0.0,www.aa.com,NaN,NaN,-0.055001,-0.109281,0.036370,0.225802,0.774164,0.810671
2,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,0.0,www.aa.com,NaN,NaN,-0.055001,-0.109281,0.036370,0.225802,0.774164,0.810671
3,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,0.0,www.aa.com,NaN,NaN,-0.043891,-0.025200,0.036016,0.135026,0.801372,0.913289
4,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,0.0,www.aa.com,NaN,NaN,-0.043891,-0.025200,0.036016,0.135026,0.801372,0.913289


In [16]:
secondstage['zscore'].describe()

count    13908.000000
mean         4.127047
std          4.160550
min        -11.967550
25%          1.870505
50%          3.215587
75%          4.857279
max         55.433156
Name: zscore, dtype: float64

In [17]:
# Read Chicago Board Momemtum Data - Volatility Index and FAMA
CBVIXFAMA = pd.read_csv('/Users/ASUS/Downloads/CBVIX.csv') 

In [18]:
secondstage = secondstage.merge(CBVIXFAMA, on = ['YEAR'], how = 'left')

In [19]:
# Create dummies for policies
# TARP's Pay Czar : 2009-2010
# Dodd Frank Act : 2010 - present
# SOX : 2002 - present

# If the year is 2009-2010, the value is 1, otherwise, it is 0
secondstage['payczar'] = np.where((secondstage['YEAR'] == 2009.0) | (secondstage['YEAR'] == 2010.0), 1, 0)

# If the year is 2010-2022, the value is 1, otherwise, it is 0
secondstage['doddfrank'] = np.where((secondstage['YEAR'] == 2010.0) | (secondstage['YEAR'] == 2011.0) | 
                                    (secondstage['YEAR'] == 2012.0) |
                                    (secondstage['YEAR'] == 2013.0) | (secondstage['YEAR'] == 2014.0) | 
                                    (secondstage['YEAR'] == 2015.0) | (secondstage['YEAR'] == 2016.0) |
                                    (secondstage['YEAR'] == 2017.0) | (secondstage['YEAR'] == 2018.0) |
                                    (secondstage['YEAR'] == 2019.0) | (secondstage['YEAR'] == 2020.0) |
                                    (secondstage['YEAR'] == 2021.0) | (secondstage['YEAR'] == 2022.0), 1, 0) 

In [37]:
secondstage.to_csv('/Users/ASUS/Downloads/zscorefinal1.csv') 

In [20]:
# Read stock data
stockreturn = pd.read_csv('/Users/ASUS/Downloads/stockreturn.csv')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10464\3355501541.py:2: DtypeWarning: Columns (6,10,25,59) have mixed types. Specify dtype option on import or set low_memory=False.
  stockreturn = pd.read_csv('/Users/ASUS/Downloads/stockreturn.csv')


In [21]:
# Rename key columns
stockreturn = stockreturn.rename(columns={"year": "YEAR", "ticker": "TICKER"})

In [24]:
# a = stockreturn.drop_duplicates(subset=['YEAR', 'TICKER'])
# a.describe()

,v1,permno,shrcd,exchcd,naics,permco,issuno,hexcd,hsiccd,dlamt,...,altprc,spread,vwretd,vwretx,ewretd,ewretx,sprtrn,YEAR,std_dev,annualized_sd
count,40694.000000,40694.000000,40691.000000,40691.000000,25601.000000,40694.000000,4.069400e+04,40694.000000,40689.000000,266.000000,...,40632.000000,436.000000,40694.000000,40694.000000,40694.000000,40694.000000,40694.000000,40694.000000,40400.000000,40400.000000
mean,83579.343466,59739.471593,12.226242,1.781033,424374.962072,22019.512164,1.127776e+06,1.761414,4969.895942,11.247013,...,46.650246,0.852036,0.008152,0.006496,0.009313,0.007730,0.006795,2007.830368,0.107257,4.966455
std,49736.153379,28385.964568,5.936854,0.982233,124504.910761,15584.047761,8.864774e+06,0.962147,2002.874005,22.042346,...,111.613881,1.534652,0.044313,0.044297,0.053290,0.053309,0.043040,8.941725,0.085942,1.537237
min,2.000000,10026.000000,10.000000,-2.000000,111339.000000,4.000000,0.000000e+00,1.000000,250.000000,0.000000,...,-798.285000,0.010000,-0.184838,-0.186323,-0.207503,-0.209773,-0.169425,1992.000000,0.000000,0.000000
25%,40904.500000,28804.000000,11.000000,1.000000,332215.000000,10352.000000,0.000000e+00,1.000000,3540.000000,0.000000,...,16.250000,0.250000,-0.018338,-0.019707,-0.019303,-0.020552,-0.017530,2000.000000,0.061685,3.942668
50%,82360.000000,75416.000000,11.000000,1.000000,424690.000000,20543.000000,4.162000e+03,1.000000,4931.000000,0.081375,...,29.655000,0.440000,0.013000,0.011479,0.013023,0.011109,0.011531,2008.000000,0.087657,4.699942
75%,122747.000000,83799.000000,11.000000,3.000000,524113.000000,30446.000000,1.746500e+04,3.000000,6351.000000,15.043750,...,50.170000,0.875000,0.035320,0.033552,0.038884,0.037053,0.034174,2016.000000,0.128252,5.685014
max,180273.000000,93436.000000,73.000000,32.000000,928110.000000,59444.000000,9.999505e+07,5.000000,9999.000000,164.970000,...,4710.930000,16.800000,0.129674,0.128408,0.225038,0.224085,0.126844,2022.000000,4.703471,34.427820


In [22]:
s = stockreturn.drop_duplicates(subset=['YEAR', 'TICKER', 'annualized_sd'])
s.describe()

,v1,permno,shrcd,exchcd,naics,permco,issuno,hexcd,hsiccd,dlamt,...,altprc,spread,vwretd,vwretx,ewretd,ewretx,sprtrn,YEAR,std_dev,annualized_sd
count,41766.000000,41766.000000,40887.000000,40887.000000,25688.000000,41766.000000,4.176600e+04,41766.000000,41760.000000,271.000000,...,41617.000000,436.000000,41766.000000,41766.000000,41766.000000,41766.000000,41766.000000,41766.000000,41424.000000,41424.000000
mean,83966.106402,59888.581095,12.224864,1.775772,424401.791264,22376.160729,1.190485e+06,1.765527,4988.381178,11.624783,...,46.069973,0.852036,0.008224,0.006568,0.009422,0.007843,0.006863,2007.731145,0.107916,4.977329
std,49925.785318,28477.188632,5.928174,0.984648,124495.785766,15783.338689,9.101293e+06,0.962941,2014.145098,22.663366,...,110.441628,1.534652,0.044196,0.044180,0.053141,0.053161,0.042941,8.961142,0.086641,1.555962
min,2.000000,10026.000000,10.000000,-2.000000,111339.000000,4.000000,0.000000e+00,1.000000,250.000000,0.000000,...,-798.285000,0.010000,-0.184838,-0.186323,-0.207503,-0.209773,-0.169425,1992.000000,0.000000,0.000000
25%,40782.250000,28564.000000,11.000000,1.000000,332214.000000,10512.000000,0.000000e+00,1.000000,3550.000000,0.000000,...,16.010000,0.250000,-0.018338,-0.019707,-0.019073,-0.020191,-0.017530,2000.000000,0.061722,3.943847
50%,83074.000000,75578.000000,11.000000,1.000000,424690.000000,20574.000000,4.202000e+03,1.000000,4931.000000,0.100000,...,29.190000,0.440000,0.013000,0.011479,0.013023,0.011109,0.011531,2008.000000,0.087987,4.708782
75%,123448.000000,83946.000000,11.000000,3.000000,524113.000000,30929.000000,1.772300e+04,3.000000,6399.000000,16.389090,...,49.610000,0.875000,0.036354,0.034641,0.038909,0.037065,0.034174,2016.000000,0.129104,5.703876
max,180273.000000,93436.000000,73.000000,32.000000,928110.000000,59444.000000,9.999505e+07,5.000000,9999.000000,164.970000,...,4710.930000,16.800000,0.129674,0.128408,0.225038,0.224085,0.126844,2022.000000,4.703471,34.427820


In [25]:
# Merge executive data with firm's financial data
stockemp = ceo_tenure.merge(df2[['YEAR', 'TICKER', 'emp']], on = ['YEAR', 'TICKER'], how = 'left')
stockemp.head()

,EXEC_FULLNAME,CFOANN,EXECDIR,OLD_DATAFMT_FLAG,INTERLOCK,REPRICE,EXECRANK,CO_PER_ROL,CONAME,TITLEANN,...,NAICS,SPINDEX,SIC,stockoptions,adjusted_stockoptions,adjusted_TC,dateceo,yearceo,tenure,emp
0,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,481111,2030,4512,9410.276,13.660537,14.808487,2003-04-25,2003.0,3.0,86.6
1,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,481111,2030,4512,9410.276,13.660537,14.808487,2003-04-25,2003.0,3.0,86.6
2,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,481111,2030,4512,9410.276,13.660537,14.808487,2003-04-25,2003.0,3.0,86.6
3,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,481111,2030,4512,5895.570,8.321363,9.657765,2003-04-25,2003.0,4.0,85.5
4,Gerard J. Arpey,NaN,1,0,0,0,NaN,14221,AMERICAN AIRLINES GROUP INC,"chairman, president & CEO",...,481111,2030,4512,5895.570,8.321363,9.657765,2003-04-25,2003.0,4.0,85.5


In [26]:
# Change merge style
stockiv = stockemp.merge(s, on = ['YEAR', 'TICKER'], how = 'left')

In [18]:
stockiv = stockiv.merge(CBVIXFAMA, on = ['YEAR'], how = 'left')
# Create dummies for policies
# TARP's Pay Czar : 2009-2010
# Dodd Frank Act : 2010 - present
# SOX : 2002 - present

# If the year is 2009-2010, the value is 1, otherwise, it is 0
stockiv['payczar'] = np.where((stockiv['YEAR'] == 2009) | (stockiv['YEAR'] == 2010), 1, 0)

# If the year is 2010-2022, the value is 1, otherwise, it is 0
stockiv['doddfrank'] = np.where((stockiv['YEAR'] == 2010) | (stockiv['YEAR'] == 2011) | 
                                    (stockiv['YEAR'] == 2012) |
                                    (stockiv['YEAR'] == 2013) | (stockiv['YEAR'] == 2014) | 
                                    (stockiv['YEAR'] == 2015) | (stockiv['YEAR'] == 2016) |
                                    (stockiv['YEAR'] == 2017) | (stockiv['YEAR'] == 2018) |
                                    (stockiv['YEAR'] == 2019) | (stockiv['YEAR'] == 2020) |
                                    (stockiv['YEAR'] == 2021) | (stockiv['YEAR'] == 2022), 1, 0) 

In [21]:
stockiv.to_csv('/Users/ASUS/Downloads/stockvolfinal1.csv') 